<a href="https://colab.research.google.com/github/sumu97/Data-Science-Cheat-Code/blob/main/Kospi_Market_Analysis_sumu97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#코스피 동향분석 및 예측 - sumu97

###개요
 - 코스피 지수의 동향을 예측하여, 선물옵션을 통한 단기적 수익실현 및 투자지표로서의 활용이 가능하도록 합니다.
 - 추후 동일한 모델을 부동산 시장 등에 적용시키기 위해, 데이터에 맞춰 쉽게 변형할 수 있도록 작성합니다.
 - 두 개 이상의 예측 모델을 활용하여 그 결과를 토대로 서로의 예측을 개선할 수 있게 합니다.
 - 아리마 분석으로 Feature 수치를 대략적으로 예측하고, 예측된 데이터를 토대로 Random Forest를 활용해 Target Value를 구합니다.

## 1. 데이터

In [1]:
%%capture
from google.colab import drive
import io
import pandas as pd
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjt8b2zg5vZTei5OWz7GTUBvV8JGu93P3pElbCTl5ee44Sr_JRFvro


In [2]:
# 데이터
import datetime
import numpy as np

def get_data(NAME,col):
  # 1. 데이터 가져오기
  df = pd.read_csv('drive/MyDrive/Project/Data_For_Kospi/' + NAME + '.csv')

  # 2. 종가, 거래량, 변동% 제외한 Feature 제거 (중복 Issue 가능성)
  df = df.drop(['오픈','고가','저가'], axis=1)

  # 3. 컬럼명 변경 (원본 데이터마다 Columns 다름, 오류처리구문 추가)
  columns_volp = ['date' , col + '_' + 'enpr' , col + '_' + 'volm' , col + '_' + 'chag']
  columns_volm = ['date' , col + '_' + 'enpr' , col + '_' + 'chag']
  try:
    df.columns = columns_volp
  except:
    df.columns = columns_volm

  # 4. 자료형 변경
  for col in df.columns:
    temp_list = []
    for i in df[col]:
      
      try:
        temp = i.replace('년','') # ER : SILVER ('sv_enpr',) 23.925 에서 에러발생 / float를 replace 하다보니 문제 / 츄라이로 넘겨서 해결
        temp = temp.replace('월','')
        temp = temp.replace('일','')
        temp = temp.replace(' ','-') # 수정 / Debug
        temp = temp.replace(',','')
        temp = temp.replace('K','000')
        temp = temp.replace('%','')
      except:
        temp_list.append(i) # 첨부터 float인 자료형을 list에 저장 / ER : append(temp) 해놓고 뭐가문젠지 한참 찾았네..
        continue # ER : 컬럼에 리스트 적용시 길이가 달라 에러발생 / 아래 츄라이문 하면서 중복적용된게 문제 / 컨티뉴로 돌려보내서 해결
        # ER : 뒷쪽 구문 먹어버려서 'sv_enpr' 에 의미가 다른 값이 들어감
      try:
        temp = float(temp)
        temp_list.append(temp) # float 로 변경된 자료형을 list에 저장
      except:
        try:
          #print(col, i, temp)
          temp = datetime.datetime.strptime(temp, '%Y-%m-%d').date()
          temp_list.append(temp) # datetime 로 변경된 자료형을 list에 저장
        except:
          temp = np.nan
          temp_list.append(temp) # NaN 으로 변경된 자료형을 list에 저장
    df[col] = temp_list # 변경된 리스트를 col 에 저장

  print(df.shape) # Debug code / 으아아 같은기간 긁었는데 뭐가이렇게 달라

  # 5. 데이터프레임으로 반환
  return df

gd = get_data('GOLD','gd') # 금 시세
sv = get_data('SILVER','sv') # 은 시세
ob = get_data('OIL_BRENT','ob') # 브렌트유 시세
ow = get_data('OIL_WTI','ow') # WTI유 시세
sp = get_data('S&P','sp') # 미국 증시 S&P 500 지수
kd = get_data('KOSDAQ','kd') # 코스닥
kp = get_data('KOSPI','kp') # 코스피(Target)
kr = get_data('KR10','kr') # 한국 10년 국채 수익률
us = get_data('US10','us') # 미국 10년 국채 수익률
jk = get_data('JPY_KRW','jk') # 엔 - 원 환율
uk = get_data('USD_KRW','uk') # 달러 - 원 환율

gd.set_index('date', inplace = True)
sv.set_index('date', inplace = True)
ob.set_index('date', inplace = True)
ow.set_index('date', inplace = True)
sp.set_index('date', inplace = True)
kd.set_index('date', inplace = True)
kp.set_index('date', inplace = True)
kr.set_index('date', inplace = True)
us.set_index('date', inplace = True)
jk.set_index('date', inplace = True)
uk.set_index('date', inplace = True)

df = gd.join(sv, how='outer').join(ob, how='outer').join(ow, how='outer').join(sp , how='outer').join(kd , how='outer').join(kp , how='outer').join(kr , how='outer').join(us , how='outer').join(jk , how='outer').join(uk , how='outer')

(2642, 4)
(3140, 4)
(2604, 4)
(2635, 4)
(2537, 4)
(2485, 4)
(2484, 4)
(3168, 3)
(3012, 3)
(2632, 3)
(2632, 3)


In [26]:
df.head()

,gd_enpr,gd_volm,gd_chag,sv_enpr,sv_volm,sv_chag,ob_enpr,ob_volm,ob_chag,ow_enpr,ow_volm,ow_chag,sp_enpr,sp_chag,kd_enpr,kd_chag,kp_enpr,kp_chag,kr_enpr,kr_chag,us_enpr,us_chag,jk_enpr,jk_chag,uk_enpr,uk_chag
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-10-04,1614.70,0.160,-2.49,29.795,0.04,-3.11,99.790,264.830,-1.89,75.670,428.110,-2.50,1123.950,2.250,436.130,-3.010,1706.190,-3.590,3.80,-3.31,1.8170,3.890,15.59240,1.150,1198.900,1.560
2011-10-05,1640.30,0.110,1.59,30.317,0.18,1.75,102.730,215.940,2.95,79.680,336.070,5.30,1144.030,1.790,421.180,-3.430,1666.520,-2.330,3.78,-0.53,1.8910,4.070,15.46080,-0.840,1187.000,-0.990
2011-10-06,1651.90,0.040,0.71,31.970,0.01,5.45,105.730,192.570,2.92,82.590,386.090,3.65,1164.970,1.830,431.180,2.370,1710.320,2.630,3.81,0.79,1.9920,5.340,15.43820,-0.150,1183.950,-0.260
2011-10-07,1634.50,0.070,-1.05,30.958,0.06,-3.17,105.880,170.250,0.14,82.980,377.540,0.47,1155.460,-0.820,442.640,2.660,1759.770,2.890,3.82,0.26,2.0750,4.170,15.26130,-1.150,1171.000,-1.090
2011-10-09,1652.05,0.065,0.55,31.685,0.06,2.35,107.415,166.435,1.52,84.195,321.185,1.70,1175.175,1.295,448.275,2.605,1763.105,1.635,3.83,0.39,2.0725,1.955,15.24885,-0.655,1169.475,-0.675


In [6]:
df.isnull().sum()

gd_enpr     869
gd_volm     980
gd_chag     869
sv_enpr     371
sv_volm    1009
sv_chag     371
ob_enpr     907
ob_volm     911
ob_chag     907
ow_enpr     876
ow_volm    1011
ow_chag     876
sp_enpr     974
sp_volm    3511
sp_chag     974
kd_enpr    1026
kd_volm    2195
kd_chag    1026
kp_enpr    1027
kp_volm    3509
kp_chag    1027
kr_enpr     343
kr_chag     343
us_enpr     499
us_chag     499
jk_enpr     879
jk_chag     879
uk_enpr     879
uk_chag     879
dtype: int64

In [11]:
# 결측치가 1500 이상인 Feature 삭제
df = df.drop(['sp_volm','kd_volm','kp_volm'], axis = 1)

In [19]:
# 나머지 결측치는 선형 보간 (interpolation)
df_intp = df.interpolate()

In [15]:
df_intp.isnull().sum()

gd_enpr    1
gd_volm    1
gd_chag    1
sv_enpr    0
sv_volm    1
sv_chag    0
ob_enpr    1
ob_volm    1
ob_chag    1
ow_enpr    1
ow_volm    1
ow_chag    1
sp_enpr    1
sp_chag    1
kd_enpr    2
kd_chag    2
kp_enpr    2
kp_chag    2
kr_enpr    2
kr_chag    2
us_enpr    1
us_chag    1
jk_enpr    1
jk_chag    1
uk_enpr    1
uk_chag    1
dtype: int64

In [22]:
# 보간 후에도 결측치 남아있는 행 삭제 (상위 2행, 시계열 데이터에 영향 주지 않음)
df = df_intp.dropna()

In [28]:
# 중복 값 있는지 확인
df.T.duplicated()

gd_enpr    False
gd_volm    False
gd_chag    False
sv_enpr    False
sv_volm    False
sv_chag    False
ob_enpr    False
ob_volm    False
ob_chag    False
ow_enpr    False
ow_volm    False
ow_chag    False
sp_enpr    False
sp_chag    False
kd_enpr    False
kd_chag    False
kp_enpr    False
kp_chag    False
kr_enpr    False
kr_chag    False
us_enpr    False
us_chag    False
jk_enpr    False
jk_chag    False
uk_enpr    False
uk_chag    False
dtype: bool